--- Day 6: Lanternfish ---

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

-    After one day, its internal timer would become 2.
-    After another day, its internal timer would become 1.
-    After another day, its internal timer would become 0.
-    After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
-    After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.

A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

3,4,3,1,2

This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:

```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```

Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?


In [ ]:
let testInput = "3,4,3,1,2"

let parse (input: string) =
    input.Split(',')
    |> Seq.map int
    |> Seq.toList

testInput |> parse

index,value
0,3
1,4
2,3
3,1
4,2


In [ ]:
let rec simulateStep (fish: list<int>) = 
   match fish with
   | head::rest -> if head = 0 then [6; 8] @ (simulateStep rest) else (head-1) :: (simulateStep rest)
   | _ -> []


testInput |> parse |> simulateStep |> simulateStep

index,value
0,1
1,2
2,1
3,6
4,8
5,0


In [ ]:
let rec simulateSteps (steps:int) (fish:list<int>) =
    if steps = 0 then 
        fish
    else
        simulateSteps (steps-1) (simulateStep fish)

testInput |> parse |> simulateSteps 18

index,value
0,6
1,8
2,1
3,1
4,3
5,0
6,2
7,2
8,4
9,6


In [ ]:
testInput |> parse |> simulateSteps 18 |> List.length

26

In [ ]:
testInput |> parse |> simulateSteps 80 |> List.length

5934

In [ ]:
let task = System.IO.File.ReadAllText "task.txt"

// nope task |> parse |> simulateSteps 80 |> List.length

In [ ]:
let groupFish (input: List<int>) =
    let groups = Array.zeroCreate<int64> 9
    input
    |> List.iter (
        fun fish -> groups[fish] <- groups[fish]+1L
    )
    groups

testInput |> parse |> groupFish

index,value
0,0
1,1
2,1
3,2
4,1
5,0
6,0
7,0
8,0


In [ ]:
let groupStep (groups: int64[]) =
    let result = Array.zeroCreate 9
    groups[1..].CopyTo( result, 0)
    result[8] <- groups[0]
    result[6] <- result[6] + groups[0]
    result

testInput |> parse |> groupFish |> groupStep |> groupStep

index,value
0,1
1,2
2,1
3,0
4,0
5,0
6,1
7,0
8,1


In [ ]:
let rec simulateGroupSteps (steps:int) (groups:int64[]) =
    if steps = 0 then 
        groups
    else
        simulateGroupSteps (steps-1) (groupStep groups)

testInput |> parse |> groupFish |> simulateGroupSteps 18 |> Array.sum

26

In [ ]:
testInput |> parse |> groupFish |> simulateGroupSteps 80 |> Array.sum

5934

In [ ]:
task |> parse |> groupFish |> simulateGroupSteps 80 |> Array.sum

396210

--- Part Two ---

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of 26984457539 lanternfish!

How many lanternfish would there be after 256 days?


In [ ]:
testInput |> parse |> groupFish |> simulateGroupSteps 256 |> Array.sum

26984457539

In [ ]:
task |> parse |> groupFish |> simulateGroupSteps 256 |> Array.sum

1770823541496